In [139]:
import pandas as pd
import time
from igdb.wrapper import IGDBWrapper
wrapper = IGDBWrapper('6a29gho2srgafvjbh6xwazv688a06a', '8hgwz56hqv8pr3nx8uz933dj2vq8y1')

In [140]:
from igdb.igdbapi_pb2 import GameResult

byte_array = b''

for i in range(0, 988):
  if i >= 987:
    limit = 426
  else:
    limit = 500

  response = wrapper.api_request(
    'games.pb', # Note the '.pb' suffix at the endpoint
    f'fields name, external_games.*, genres.name, involved_companies, keywords.name, platforms.name, rating, rating_count, release_dates.date, category;\
      limit {limit}; offset {i * 500};'
  )

  byte_array += response

  time.sleep(0.3)

games_message = GameResult()
games_message.ParseFromString(byte_array) # Fills the protobuf message object with the response
games = games_message.games
del byte_array

In [141]:
def category_str(value):
    strings = [
        "main_game",
        "dlc_addon",
        "expansion",
        "bundle",
        "standalone_expansion",
        "mod",
        "episode",
        "season",
        "remake",
        "remaster",
        "expanded_game",
        "port",
        "fork",
        "pack",
        "update"
    ]
    return strings[value]

def platform_str(value):
    strings = {
        "1":    "steam",
        "5":    "gog",
        "10":	"youtube",
        "11":	"microsoft",
        "13":	"apple",
        "14":	"twitch",
        "15":	"android",
        "20":	"amazon_asin",
        "22":	"amazon_luna",
        "23":	"amazon_adg",
        "26":	"epic_game_store",
        "28":	"oculus",
        "29":	"utomik",
        "30":	"itch_io",
        "31":	"xbox_marketplace",
        "32":	"kartridge",
        "36":	"playstation_store_us",
        "37":	"focus_entertainment",
        "54":	"xbox_game_pass_ultimate_cloud",
        "55":	"gamejolt"
    }
    try:
        return strings[str(value)]
    except KeyError:
        return ""

def parse_games(games):
    for gm in games:
        yield {
            'name': gm.name,
            'external_games': gm.external_games,
            'genres': gm.genres,
            'involved_companies': gm.involved_companies,
            'keywords': gm.keywords,
            'platforms': gm.platforms,
            'rating': gm.rating,
            'rating_count': gm.rating_count,
            'release_dates': gm.release_dates,
            'category': category_str(gm.category)
            }

def parse_index(games):
    for gm in games:
        yield gm.id

# Creates one-to-many dataframes for game fields with more than one value
def unravel(games, field1, field2, col_name):
    df_data = []

    for gm in games:
        id = gm.id
        field = getattr(gm, field1)

        for f in field:
            df_data.append([id, getattr(f, field2)])
    
    return pd.DataFrame(df_data, columns=['game_id', col_name])

def unravel_external_games(games):
    df_data = []

    for gm in games:
        id = gm.id
        ext = gm.external_games
        
        for e in ext:
            platform = platform_str(e.category)
            if platform != "":
                df_data.append([id, e.uid, platform_str(e.category)])
    
    return pd.DataFrame(df_data, columns=['game_id', 'uid', 'platform'])

def unravel_release_dates(games):
    df_data = []

    for gm in games:
        id = gm.id
        release_dates = gm.release_dates

        for rd in release_dates:
            df_data.append([id, rd.date.seconds])
    
    return pd.DataFrame(df_data, columns=['game_id', 'release_date'])

def unravel_companies(games):
    df_data = []

    for gm in games:
        id = gm.id
        comp = gm.involved_companies

        for c in comp:
            df_data.append([id, c.id])
    
    return pd.DataFrame(df_data, columns=['game_id', 'company_id'])

In [142]:
df = pd.DataFrame(parse_games(games), index=parse_index(games))
df

,name,external_games,genres,involved_companies,keywords,platforms,rating,rating_count,release_dates,category
231577,Blood Bowl 3: Black Orcs Edition,[id: 2639959\ncategory: EXTERNALGAME_EPIC_GAME...,"[id: 15\nname: ""Strategy""\n]",[],[],"[id: 49\nname: ""Xbox One""\n, id: 169\nname: ""X...",0.000000,0,[id: 455316\ndate {\n seconds: 1677110400\n}\...,main_game
147666,Shinobi Blade,[id: 2011978\ncategory: EXTERNALGAME_TWITCH\nc...,"[id: 4\nname: ""Fighting""\n, id: 31\nname: ""Adv...",[],[],"[id: 130\nname: ""Nintendo Switch""\n]",0.000000,0,[id: 249798\ndate {\n seconds: 1611014400\n}\...,main_game
44711,DDRMax2: Dance Dance Revolution,[id: 154169\ncategory: 3\ncreated_at {\n seco...,"[id: 7\nname: ""Music""\n, id: 13\nname: ""Simula...","[id: 102978\n, id: 102979\n]","[id: 1699\nname: ""dancing""\n, id: 2152\nname: ...","[id: 8\nname: ""PlayStation 2""\n, id: 52\nname:...",0.000000,0,[id: 201923\ndate {\n seconds: 1019001600\n}\...,main_game
85450,Transformers Prime: The Game,[id: 1991656\ncategory: EXTERNALGAME_TWITCH\nc...,[],[id: 85230\n],[],[],0.000000,0,[],main_game
95080,Dotra,[id: 1989881\ncategory: EXTERNALGAME_TWITCH\nc...,[],[],[],[],0.000000,0,[],main_game
...,...,...,...,...,...,...,...,...,...,...
260221,Smoots Pinball,[id: 2729371\ncategory: EXTERNALGAME_STEAM\ncr...,"[id: 13\nname: ""Simulator""\n, id: 14\nname: ""S...",[id: 240194\n],"[id: 934\nname: ""pinball""\n, id: 4456\nname: ""...","[id: 6\nname: ""PC (Microsoft Windows)""\n, id: ...",0.000000,0,[id: 534596\ndate {\n seconds: 1700092800\n}\...,main_game
121481,Tasomachi: Behind the Twilight,[id: 1731429\ncategory: EXTERNALGAME_STEAM\ncr...,"[id: 8\nname: ""Platform""\n, id: 31\nname: ""Adv...","[id: 138728\n, id: 190123\n]","[id: 78\nname: ""anime""\n]","[id: 6\nname: ""PC (Microsoft Windows)""\n, id: ...",0.000000,0,[id: 238218\ndate {\n seconds: 1618358400\n}\...,main_game
187244,Pocket Soccer,[id: 2223453\ncategory: EXTERNALGAME_TWITCH\nc...,"[id: 14\nname: ""Sport""\n, id: 33\nname: ""Arcad...",[],[],"[id: 48\nname: ""PlayStation 4""\n, id: 130\nnam...",0.000000,0,[id: 337193\ndate {\n seconds: 1634256000\n}\...,main_game
4796,de Blob 2,[id: 71685\ncategory: EXTERNALGAME_STEAM\ncrea...,"[id: 8\nname: ""Platform""\n, id: 9\nname: ""Puzz...","[id: 11546\n, id: 11547\n, id: 11548\n, id: 11...","[id: 589\nname: ""2.5d""\n, id: 824\nname: ""colo...","[id: 5\nname: ""Wii""\n, id: 6\nname: ""PC (Micro...",70.169141,17,[id: 11000\ndate {\n seconds: 1298332800\n}\n...,main_game


In [143]:
external_games = unravel_external_games(games)
genres = unravel(games, 'genres', 'name', 'genre')
involved_companies = unravel_companies(games)
keywords = unravel(games, 'keywords', 'name', 'keyword')
platforms = unravel(games, 'platforms', 'name', 'platform')
release_dates = unravel_release_dates(games)
df = df[['name','rating','rating_count','category']]

In [145]:
df.to_csv('data/igdb.csv', index_label='game_id')
genres.to_csv('data/igdb_genres.csv', index=False)
external_games.to_csv('data/igdb_external_games.csv', index=False)
involved_companies.to_csv('data/igdb_involved_companies.csv', index=False)
keywords.to_csv('data/igdb_keywords.csv', index=False)
platforms.to_csv('data/igdb_platforms.csv', index=False)
release_dates.to_csv('data/igdb_release_dates.csv', index=False)

In [156]:
from igdb.igdbapi_pb2 import CompanyResult

byte_array = b''

for i in range(0, 98):
  if i >= 97:
    limit = 255
  else:
    limit = 500

  response = wrapper.api_request(
    'companies.pb', # Note the '.pb' suffix at the endpoint
    f'fields name, developed;\
      limit {limit}; offset {i * 500};'
  )

  byte_array += response

  time.sleep(0.3)

companies_message = CompanyResult()
companies_message.ParseFromString(byte_array) # Fills the protobuf message object with the response
companies = companies_message.companies
del byte_array

In [161]:
def parse_companies(companies):
    for com in companies:
        yield {
            "name": com.name,
            "developed": com.developed
            }

def parse_companies_index(companies):
    for com in companies:
        yield com.id

def unravel_com_df(comapnies):
    df_data = []

    for com in companies:
        id = com.id
        developed = com.developed

        for dev in developed:
            df_data.append([id, dev.id])
    
    return pd.DataFrame(df_data, columns=['company_id', 'game_id'])

In [158]:
com_df = pd.DataFrame(parse_companies(companies), index=parse_companies_index(companies))
com_df

,name,developed
38808,puyofan99,[]
15538,TDZ Games,[id: 32277\n]
21167,SeaDads,"[id: 120628\n, id: 137732\n]"
42407,Abracadabra Games,[id: 210521\n]
5807,Creative Services EARS,[]
...,...,...
13956,Namco Limited,"[id: 2710\n, id: 2751\n, id: 18620\n, id: 2012..."
837,LEVEL-5,"[id: 1214\n, id: 1215\n, id: 1397\n, id: 1398\..."
1743,ASCII Corporation,"[id: 3283\n, id: 6102\n, id: 19493\n, id: 1955..."
3455,.theprodukkt,"[id: 7724\n, id: 94683\n]"


In [160]:
com_df["name"].to_csv('data/igdb_companies.csv', index_label='company_id')

In [162]:
dev_df = unravel_com_df(com_df)
dev_df

,company_id,game_id
0,15538,32277
1,21167,120628
2,21167,137732
3,42407,210521
4,39245,173242
...,...,...
105788,765,133923
105789,765,140611
105790,765,152358
105791,765,217550


In [163]:
dev_df.to_csv('data/igdb_developed.csv', index=False)